# **Task 1: Find-S Algorithm**

In [4]:
import pandas as pd

# Dataset
data = pd.DataFrame([
    ["Sunny","Warm","Normal","Strong","Warm","Same","Yes"],
    ["Sunny","Warm","High","Strong","Warm","Same","Yes"],
    ["Rainy","Cold","High","Strong","Warm","Change","No"],
    ["Sunny","Warm","High","Strong","Cool","Change","Yes"]
], columns=["Sky","AirTemp","Humidity","Wind","Water","Forecast","EnjoySport"])

attributes = ["Sky","AirTemp","Humidity","Wind","Water","Forecast"]
target = "EnjoySport"

# Find-S Algorithm
def find_s_algorithm(df, attributes, target):
    hypothesis = ['Ø'] * len(attributes)
    print("Initial Hypothesis:", hypothesis)

    for index, row in df.iterrows():
        if row[target] == 'Yes':
            for i, attr in enumerate(attributes):
                if hypothesis[i] == 'Ø':
                    hypothesis[i] = row[attr]
                elif hypothesis[i] != row[attr]:
                    hypothesis[i] = '?'
            print(f"After positive example {index+1}: {hypothesis}")

    return hypothesis

print("\nRunning Find-S Algorithm...")
final_hypothesis = find_s_algorithm(data, attributes, target)
print("\nFinal Hypothesis:", final_hypothesis)



Running Find-S Algorithm...
Initial Hypothesis: ['Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø']
After positive example 1: ['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same']
After positive example 2: ['Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same']
After positive example 4: ['Sunny', 'Warm', '?', 'Strong', '?', '?']

Final Hypothesis: ['Sunny', 'Warm', '?', 'Strong', '?', '?']


# **Task 2: Candidate Elimination Algorithm**

In [5]:
def candidate_elimination(df, attributes, target):
    # Initialize S and G
    S = [['Ø'] * len(attributes)]
    G = [['?'] * len(attributes)]

    def is_more_general(h1, h2):
        more_general = False
        for x, y in zip(h1, h2):
            if x == '?':
                if y != '?':
                    more_general = True
            elif x != y and y != 'Ø':
                return False
        return more_general

    print("\nInitial S:", S)
    print("Initial G:", G)

    for index, row in df.iterrows():
        example = row[attributes].values
        if row[target] == 'Yes':
            G = [g for g in G if all(g[i] == '?' or g[i] == example[i] for i in range(len(attributes)))]
            for s in S:
                for i in range(len(attributes)):
                    if s[i] == 'Ø':
                        s[i] = example[i]
                    elif s[i] != example[i]:
                        s[i] = '?'
        else:
            S = [s for s in S if not all(s[i] == '?' or s[i] == example[i] for i in range(len(attributes)))]
            new_G = []
            for g in G:
                if all(g[i] == '?' or g[i] == example[i] for i in range(len(attributes))):
                    for i in range(len(attributes)):
                        if g[i] == '?':
                            for val in df[attributes[i]].unique():
                                if val != example[i]:
                                    new_hypothesis = g.copy()
                                    new_hypothesis[i] = val
                                    new_G.append(new_hypothesis)
                else:
                    new_G.append(g)
            G = new_G

        print(f"\nAfter example {index+1} ({row[target]}):")
        print("S:", S)
        print("G:", G)

    return S, G

print("\nRunning Candidate Elimination Algorithm...")
S_final, G_final = candidate_elimination(data, attributes, target)
print("\nFinal S:", S_final)
print("Final G:", G_final)



Running Candidate Elimination Algorithm...

Initial S: [['Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø']]
Initial G: [['?', '?', '?', '?', '?', '?']]

After example 1 (Yes):
S: [['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same']]
G: [['?', '?', '?', '?', '?', '?']]

After example 2 (Yes):
S: [['Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same']]
G: [['?', '?', '?', '?', '?', '?']]

After example 3 (No):
S: [['Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same']]
G: [['Sunny', '?', '?', '?', '?', '?'], ['?', 'Warm', '?', '?', '?', '?'], ['?', '?', 'Normal', '?', '?', '?'], ['?', '?', '?', '?', 'Cool', '?'], ['?', '?', '?', '?', '?', 'Same']]

After example 4 (Yes):
S: [['Sunny', 'Warm', '?', 'Strong', '?', '?']]
G: [['Sunny', '?', '?', '?', '?', '?'], ['?', 'Warm', '?', '?', '?', '?'], ['?', '?', '?', '?', 'Cool', '?']]

Final S: [['Sunny', 'Warm', '?', 'Strong', '?', '?']]
Final G: [['Sunny', '?', '?', '?', '?', '?'], ['?', 'Warm', '?', '?', '?', '?'], ['?', '?', '?', '?', 'Cool', '?']]


# **Task 3: Test with New Dataset & Noise**

In [6]:
# New dataset with noise
new_data = pd.DataFrame([
    ["Sunny","Warm","Normal","Strong","Warm","Same","Yes"],
    ["Sunny","Warm","High","Strong","Warm","Same","Yes"],
    ["Rainy","Cold","High","Strong","Warm","Change","No"],
    ["Sunny","Warm","High","Strong","Cool","Change","Yes"],
    ["Sunny","Cold","Normal","Weak","Cool","Same","No"],
    ["Rainy","Warm","High","Strong","Cool","Same","Yes"],  # noisy/contradictory
], columns=["Sky","AirTemp","Humidity","Wind","Water","Forecast","EnjoySport"])

print("\nRunning Find-S on New Dataset...")
find_s_result = find_s_algorithm(new_data, attributes, target)

print("\nRunning Candidate Elimination on New Dataset...")
S_noise, G_noise = candidate_elimination(new_data, attributes, target)

print("\nFinal S with Noise:", S_noise)
print("Final G with Noise:", G_noise)



Running Find-S on New Dataset...
Initial Hypothesis: ['Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø']
After positive example 1: ['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same']
After positive example 2: ['Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same']
After positive example 4: ['Sunny', 'Warm', '?', 'Strong', '?', '?']
After positive example 6: ['?', 'Warm', '?', 'Strong', '?', '?']

Running Candidate Elimination on New Dataset...

Initial S: [['Ø', 'Ø', 'Ø', 'Ø', 'Ø', 'Ø']]
Initial G: [['?', '?', '?', '?', '?', '?']]

After example 1 (Yes):
S: [['Sunny', 'Warm', 'Normal', 'Strong', 'Warm', 'Same']]
G: [['?', '?', '?', '?', '?', '?']]

After example 2 (Yes):
S: [['Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same']]
G: [['?', '?', '?', '?', '?', '?']]

After example 3 (No):
S: [['Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same']]
G: [['Sunny', '?', '?', '?', '?', '?'], ['?', 'Warm', '?', '?', '?', '?'], ['?', '?', 'Normal', '?', '?', '?'], ['?', '?', '?', 'Weak', '?', '?'], ['?', '?', '?', '?', 'Cool', '?